<a href="https://colab.research.google.com/github/min1112/Data_Analysis_Skill_Exercise/blob/main/RFM_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

패키지 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

깃헙에서 CSV 파일 불러오기

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/min1112/Data_Analysis_Skill_Exercise/main/Part3.ch03_RFM/rfm_mart.csv')
df

,mem_no,last_ord_dt,recency,frequency,monetary,is_back
0,1,2023-01-01,30,1,28700,0
1,2,2023-01-12,19,7,26929,0
2,3,2023-01-01,30,1,54800,0
3,4,2023-01-27,4,3,25700,1
4,5,2023-01-01,30,2,6500,0
...,...,...,...,...,...,...
995,996,2023-01-02,29,3,31000,1
996,997,2023-01-30,1,2,16450,1
997,998,2023-01-01,30,1,13600,0
998,999,2023-01-01,30,1,14200,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mem_no       1000 non-null   int64 
 1   last_ord_dt  1000 non-null   object
 2   recency      1000 non-null   int64 
 3   frequency    1000 non-null   int64 
 4   monetary     1000 non-null   int64 
 5   is_back      1000 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 47.0+ KB


로지스틱 회귀계수 구하기

In [ ]:
X = df.drop(['mem_no', 'last_ord_dt','is_back'], axis=1)
y = df['is_back']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
model.score(X_test, y_test)

0.724

In [ ]:
coef = pd.DataFrame({'features':X.columns, 'coefficient':model.coef_[0]})
coef

,features,coefficient
0,recency,-0.048644
1,frequency,0.007649
2,monetary,0.000021


In [ ]:
coef.iloc[1,1]

0.00764929960049877

백분위수(Percentile) 기반으로 RFM 점수 산출하기

In [ ]:
a1, a2, a3 = np.percentile(df['recency'], [20,40,60])
a1, a2, a3

(7.0, 23.0, 30.0)

In [ ]:
def percent(x) :
  if x <= a1 :
    return 4
  elif x > a1 and x <= a2 :
    return 3
  elif x > a2 and x < a3 :
    return 2
  elif x >= a3 :
    return 1

In [ ]:
df['recency_score'] = df['recency'].apply(percent)*-coef.iloc[0,1]
df

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score
0,1,2023-01-01,30,1,28700,0,0.048644
1,2,2023-01-12,19,7,26929,0,0.145931
2,3,2023-01-01,30,1,54800,0,0.048644
3,4,2023-01-27,4,3,25700,1,0.194575
4,5,2023-01-01,30,2,6500,0,0.048644
...,...,...,...,...,...,...,...
995,996,2023-01-02,29,3,31000,1,0.097288
996,997,2023-01-30,1,2,16450,1,0.194575
997,998,2023-01-01,30,1,13600,0,0.048644
998,999,2023-01-01,30,1,14200,1,0.048644


In [ ]:
b1, b2, b3 = np.percentile(df['frequency'], [10,50,80])
b1, b2, b3

(1.0, 2.0, 3.0)

In [ ]:
def percent(x) :
  if x <= b1 :
    return 1
  elif x > b1 and x <= b2 :
    return 2
  elif x > b2 and x <= b3 :
    return 3
  elif x > b3 :
    return 4

In [ ]:
df['frequency_score'] = df['frequency'].apply(percent)*coef.iloc[1,1]
df

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score
0,1,2023-01-01,30,1,28700,0,0.048644,0.007649
1,2,2023-01-12,19,7,26929,0,0.145931,0.030597
2,3,2023-01-01,30,1,54800,0,0.048644,0.007649
3,4,2023-01-27,4,3,25700,1,0.194575,0.022948
4,5,2023-01-01,30,2,6500,0,0.048644,0.015299
...,...,...,...,...,...,...,...,...
995,996,2023-01-02,29,3,31000,1,0.097288,0.022948
996,997,2023-01-30,1,2,16450,1,0.194575,0.015299
997,998,2023-01-01,30,1,13600,0,0.048644,0.007649
998,999,2023-01-01,30,1,14200,1,0.048644,0.007649


In [ ]:
c1, c2, c3 = np.percentile(df['monetary'], [20,40,80])
c1, c2, c3

(18500.0, 22299.800000000003, 34020.00000000001)

In [ ]:
def percent(x) :
  if x <= c1 :
    return 1
  elif x > c1 and x <= c2 :
    return 2
  elif x > c2 and x <= c3 :
    return 3
  elif x > c3 :
    return 4

In [ ]:
df['monetary_score'] = df['monetary'].apply(percent)*coef.iloc[2,1]
df

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score
0,1,2023-01-01,30,1,28700,0,0.048644,0.007649,0.000063
1,2,2023-01-12,19,7,26929,0,0.145931,0.030597,0.000063
2,3,2023-01-01,30,1,54800,0,0.048644,0.007649,0.000084
3,4,2023-01-27,4,3,25700,1,0.194575,0.022948,0.000063
4,5,2023-01-01,30,2,6500,0,0.048644,0.015299,0.000021
...,...,...,...,...,...,...,...,...,...
995,996,2023-01-02,29,3,31000,1,0.097288,0.022948,0.000063
996,997,2023-01-30,1,2,16450,1,0.194575,0.015299,0.000021
997,998,2023-01-01,30,1,13600,0,0.048644,0.007649,0.000021
998,999,2023-01-01,30,1,14200,1,0.048644,0.007649,0.000021


In [ ]:
df['total_score'] = df['recency_score'] + df['frequency_score'] + df['monetary_score']
df

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score
0,1,2023-01-01,30,1,28700,0,0.048644,0.007649,0.000063,0.056356
1,2,2023-01-12,19,7,26929,0,0.145931,0.030597,0.000063,0.176591
2,3,2023-01-01,30,1,54800,0,0.048644,0.007649,0.000084,0.056377
3,4,2023-01-27,4,3,25700,1,0.194575,0.022948,0.000063,0.217586
4,5,2023-01-01,30,2,6500,0,0.048644,0.015299,0.000021,0.063963
...,...,...,...,...,...,...,...,...,...,...
995,996,2023-01-02,29,3,31000,1,0.097288,0.022948,0.000063,0.120298
996,997,2023-01-30,1,2,16450,1,0.194575,0.015299,0.000021,0.209895
997,998,2023-01-01,30,1,13600,0,0.048644,0.007649,0.000021,0.056314
998,999,2023-01-01,30,1,14200,1,0.048644,0.007649,0.000021,0.056314


총 점수로 5그룹 분류

In [ ]:
t1, t2, t3, t4 = np.percentile(df['total_score'], [20,50,70,90])
t1, t2, t3, t4

(0.05635605447986593,
 0.0640263537891815,
 0.16892116547788022,
 0.2176069206484306)

In [ ]:
def level(x) :
  if x <= t1 :
    return 5
  elif x > t1 and x <= t2 :
    return 4
  elif x > t2 and x <= t3 :
    return 3
  elif x > t3 and x <= t4 :
    return 2
  elif x > t4 :
    return 1

In [ ]:
df['level'] = df['total_score'].apply(level)
df

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score,level
0,1,2023-01-01,30,1,28700,0,0.048644,0.007649,0.000063,0.056356,5
1,2,2023-01-12,19,7,26929,0,0.145931,0.030597,0.000063,0.176591,2
2,3,2023-01-01,30,1,54800,0,0.048644,0.007649,0.000084,0.056377,4
3,4,2023-01-27,4,3,25700,1,0.194575,0.022948,0.000063,0.217586,2
4,5,2023-01-01,30,2,6500,0,0.048644,0.015299,0.000021,0.063963,4
...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-01-02,29,3,31000,1,0.097288,0.022948,0.000063,0.120298,3
996,997,2023-01-30,1,2,16450,1,0.194575,0.015299,0.000021,0.209895,2
997,998,2023-01-01,30,1,13600,0,0.048644,0.007649,0.000021,0.056314,5
998,999,2023-01-01,30,1,14200,1,0.048644,0.007649,0.000021,0.056314,5


In [ ]:
df.level.value_counts()

5    359
3    205
2    194
4    143
1     99
Name: level, dtype: int64

5그룹별로 리텐션(1-리텐션) 구하기

In [ ]:
pivot = df.groupby('level').agg({'is_back':'sum', 'mem_no':'count'})
pivot

,is_back,mem_no
level,,
1,88,99
2,106,194
3,91,205
4,36,143
5,97,359


In [ ]:
pivot['retention'] = pivot.is_back/pivot.mem_no
pivot

,is_back,mem_no,retention
level,,,
1,88,99,0.888889
2,106,194,0.546392
3,91,205,0.443902
4,36,143,0.251748
5,97,359,0.270195


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(pivot, *['is_back'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(pivot, *['mem_no'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(pivot, *['retention'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(pivot, *['is_back'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(pivot, *['retention'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(pivot, *['is_back'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(pivot, *['mem_no'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(pivot, *['retention'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(pivot, *[[['is_back', 'mem_no'], ['mem_no', 'retention']]], **{})
chart